# Lesson 1: Advanced RAG Pipeline

In [1]:
import utils
from utils import get_prebuilt_trulens_recorder
import os
import openai
openai.api_key = utils.get_openai_api_key()
print("OK")

Arguments ['max_retries'] are ignored as `client` was provided.


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
OK


[nltk_data] Downloading package punkt to /home/ncs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[6])

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'>
Doc ID: 7fe97ff5-30e0-4a24-a5d8-bc51e2faf212
Text: PAGE 7 These phases apply in a wide  range of professions, but
AI  involves unique elements. For example: Three Steps to Career
Growth AI is nascent, and many technologies are still evolving. While
the  foundations of machine learning and deep learning are maturing —
and coursework is an efficient way to master them — beyond these
foundations,...


## Basic RAG pipeline

In [3]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [4]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings

# 配置Azure OpenAI服务
azure_endpoint = "https://4-intern.openai.azure.com/"
api_key = "cda7cc8aac35491d8f6d4dd600655e62"
api_version = "2024-02-01"

llm = AzureOpenAI(
    deployment_name="agent",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)
embed_model = AzureOpenAIEmbedding(
    deployment_name="embedding",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model

index = VectorStoreIndex.from_documents([document])

In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

To build your experience through projects, you can take the following steps:

1. Join existing projects by asking to collaborate with someone who already has an idea.
2. Engage in continuous learning by reading, taking courses, and talking to domain experts to generate new ideas.
3. Focus on specific application areas where machine learning has not yet been widely applied.
4. Develop a side hustle that can stir creativity and strengthen bonds with collaborators.
5. Ensure the project helps you grow technically by being challenging but achievable.
6. Work with good teammates or find people to discuss ideas with.
7. Choose projects that can serve as stepping stones to larger, more complex projects.
8. Avoid analysis paralysis by making quick decisions on smaller projects to gain experience faster.
9. Use the "Ready, Fire, Aim" approach for low-cost, iterative projects to discover and refine ideas quickly.
10. Build a portfolio of projects that shows skill progression from simple to compl

## Evaluation setup using TruLens

In [5]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [9]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [6]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [11]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

NameError: name 'query_engine' is not defined

In [13]:
with tru_recorder as recording:
    for question in eval_questions[-2:]:
        response = query_engine.query(question)

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/5 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

app_id                                           app_json  \
0  Direct Query Engine  {"tru_class_info": {"name": "TruLlama", "modul...   
1  Direct Query Engine  {"tru_class_info": {"name": "TruLlama", "modul...   

                                                type  \
0  RetrieverQueryEngine(llama_index.core.query_en...   
1  RetrieverQueryEngine(llama_index.core.query_en...   

                                      record_id  \
0  record_hash_cd5705d4ffb72e672f4fd7e8b31cea10   
1  record_hash_d69873af259b296ffdd543946268aaec   

                                             input  \
0  "Is it normal to find parts of AI challenging?"   
1               "What is the right AI job for me?"   

                                              output tags  \
0  "Yes, it is normal to find parts of AI challen...    -   
1  "The right AI job for you depends on your curr...    -   

                                         record_json  \
0  {"record_id": "record_hash_cd5705d4ffb72e672f4...   
1  {"record_id": "record_hash_d69873af259b296ffdd...   

                                           cost_json  \
0  {"n_requests": 0, "n_successful_requests": 0, ...   
1  {"n_requests": 0, "n_successful_requests": 0, ...   

                                           perf_json  \
0  {"start_time": "2024-11-29T16:17:01.139486", "...   
1  {"start_time": "2024-11-29T16:17:05.771259", "...   

                           ts  Answer Relevance  Context Relevance  \
0  2024-11-29T16:17:05.637240               1.0               0.15   
1  2024-11-29T16:17:09.577823               1.0               0.20   

   Groundedness                             Answer Relevance_calls  \
0          0.60  [{'args': {'prompt': 'Is it normal to find par...   
1          0.76  [{'args': {'prompt': 'What is the right AI job...   

                             Context Relevance_calls  \
0  [{'args': {'prompt': 'Is it normal to find par...   
1  [{'args': {'prompt': 'What is the right AI job...   

                                  Groundedness_calls  latency  total_tokens  \
0  [{'args': {'source': 'This approach may be hel...        4             0   
1  [{'args': {'source': 'Successes build on each ...        3             0   

   total_cost  
0         0.0  
1         0.0

In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.9.127:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [ ]:
#from llama_index.llms import OpenAI

#llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [5]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model=embed_model,
    save_dir="sentence_index"
)

In [6]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [7]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

To get started on a personal project in AI, you can begin by re-implementing a known algorithm or applying machine learning to a hobby, such as predicting the outcome of your favorite sports team. You might also consider building a small but useful system at work, like a machine learning-based script to automate tasks. Participating in competitions, such as those organized by Kaggle, is another excellent way to gain experience and build your skills.


In [13]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [16]:
for question in eval_questions[-2:]:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are some common challenges in AI?
Some common challenges in AI include finding suitable projects, estimating project timelines and return on investment, setting expectations, and managing the highly iterative nature of AI projects. Additionally, there are difficulties in collaborating with stakeholders who may lack expertise in AI, and dealing with inconsistent opinions on AI skills and job roles.


Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging. Many people, including those who have published significant AI research, have struggled with technical challenges at some point.


Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
tru.get_leaderboard(app_ids=[])

Groundedness  Answer Relevance  \
app_id                                                         
Sentence Window Query Engine           1.0              0.95   

                              Context Relevance  latency  total_cost  
app_id                                                                
Sentence Window Query Engine              0.725      2.5         0.0

In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

### 2. Auto-merging retrieval

In [7]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model=embed_model,
    #embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [8]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [9]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 8b010d33-dfa1-4afa-a81d-ab22e30b2dac.
> Parent node text: PAGE 21
Building a Portfolio of 
Projects that Shows 
Skill Progression 
CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: 5cc9c2a1-95ec-4588-8033-dca5adf70b49.
> Parent node text: PAGE 21
Building a Portfolio of 
Projects that Shows 
Skill Progression 
CHAPTER 6
PROJECTS

To build a portfolio of AI projects, start by working on a variety of projects that show progress over time, from simple to more complex undertakings. Focus on identifying ideas that are worth working on and apply machine learning to different industries such as manufacturing, healthcare, climate change, agriculture, ecommerce, and advertising. Additionally, ensure you can effectively communicate your thinking and the value of your work to others. This will help you gain trust and resources for larger projects.


In [10]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [11]:
for question in eval_questions[-2:]:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

> Merging 4 nodes into parent node.
> Parent node id: 53bae10f-a56b-4958-ad69-20fdd3e838d6.
> Parent node text: PAGE 7
These phases apply in a wide 
range of professions, but AI 
involves unique elements.
For ...

> Merging 4 nodes into parent node.
> Parent node id: 425579a8-a72e-4753-badc-bc94eb7c04b7.
> Parent node text: PAGE 15
One of the most important skills of an AI architect is the ability to identify ideas that...

> Merging 1 nodes into parent node.
> Parent node id: 1ec6dfdc-991d-4980-80f4-299fac5879ef.
> Parent node text: PAGE 7
These phases apply in a wide 
range of professions, but AI 
involves unique elements.
For ...

> Merging 1 nodes into parent node.
> Parent node id: 5dedfbf2-b487-4553-86e6-3d4934bf129c.
> Parent node text: PAGE 15
One of the most important skills of an AI architect is the ability to identify ideas that...

What are some common challenges in AI?
Some common challenges in AI include finding suitable projects, estimating project timelines and return o

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

> Merging 4 nodes into parent node.
> Parent node id: ecb76795-173d-4f8e-a4c4-37536d27c34b.
> Parent node text: PAGE 38
Before we dive into the final chapter of this book, I’d like to address the serious matte...

> Merging 1 nodes into parent node.
> Parent node id: 6402085f-8a60-4080-aa8d-c6cdb884b522.
> Parent node text: I’ve found EDA particularly 
useful in data-centric AI development, where analyzing errors and ga...

> Merging 3 nodes into parent node.
> Parent node id: 53bae10f-a56b-4958-ad69-20fdd3e838d6.
> Parent node text: PAGE 7
These phases apply in a wide 
range of professions, but AI 
involves unique elements.
For ...

> Merging 1 nodes into parent node.
> Parent node id: bd842a69-f0e5-43de-8fe6-9533c6d48be9.
> Parent node text: PAGE 38
Before we dive into the final chapter of this book, I’d like to address the serious matte...

> Merging 1 nodes into parent node.
> Parent node id: 1ec6dfdc-991d-4980-80f4-299fac5879ef.
> Parent node text: PAGE 7
These phases apply in a w

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
tru.get_leaderboard(app_ids=[])

Groundedness  Answer Relevance  Context Relevance  \
app_id                                                                        
Automerging Query Engine           1.0               1.0               0.45   

                          latency  total_cost  
app_id                                         
Automerging Query Engine      2.5         0.0

In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()